**Introduction**

In this data analysis report, I did feature visualization and selection as a different from other kernels. Feature selection with correlation, univariate feature selection, recursive feature elimination, recursive feature elimination with cross validation and tree based feature selection methods are used with random forest classification. Also, principle component analysis are used to observe number of components.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
import time
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [ ]:
data=pd.read_csv('../input/breast-cancer-wisconsin-data/data.csv')

In [ ]:
data.head(5)

In [ ]:
data.columns

In [ ]:
y=data['diagnosis']

In [ ]:
list=['id','diagnosis','Unnamed: 32']
X=data.drop(list,axis=1)

In [ ]:
X.head()

In [ ]:
str(len(X.columns))

In [ ]:
ax=sns.countplot(y)
B,M=y.value_counts()
print('Benign={} and  Malignant= {}'.format(B,M))

In [ ]:
X.describe()


In [ ]:
data1=X
data_n= (data1-data1.mean())/(data1.std())
data2=pd.concat([y,data_n.iloc[:,0:10]],axis=1)
data2=pd.melt(data2,id_vars="diagnosis",var_name="features",value_name='value')

plt.figure(figsize=(10,10))
sns.violinplot(x="features", y="value", hue="diagnosis", data=data2, split=True, inner="quart")
plt.xticks(rotation=90)



In [ ]:
plt.figure(figsize=(10,10))
sns.boxplot(x="features",y="value", hue="diagnosis", data=data2)
plt.xticks(rotation=90)

In [ ]:
sns.jointplot(X.loc[:,'concavity_worst'],X.loc[:,'concave points_worst'],kind="reg", color="#ce1433")

In [ ]:
sns.set(style="whitegrid", palette="muted")
data_dia=y
data=X
data_n_2= (data-data.mean())/(data.std())
data=pd.concat([y, data_n_2.iloc[:,0:10]], axis=1)
data=pd.melt(data, id_vars="diagnosis", var_name="features", value_name='value')
plt.figure(figsize=(10,10))
tic=time.time()
sns.swarmplot(x="features", y="value", hue="diagnosis", data=data)
plt.xticks(rotation=90)

In [ ]:
drop_list1 = ['perimeter_mean','radius_mean','compactness_mean','concave points_mean','radius_se','perimeter_se','radius_worst','perimeter_worst','compactness_worst','concave points_worst','compactness_se','concave points_se','texture_worst','area_worst']
x_1=X.drop(drop_list1, axis=1)
x_1.head()


In [ ]:
f,ax=plt.subplots(figsize=(14,14))
sns.heatmap(x_1.corr(),annot=True, linewidths=5, fmt='.1f', ax=ax)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_1, y, test_size=0.3, random_state=40)

In [ ]:
model= RandomForestClassifier(random_state=40)
model.fit(X_train, y_train)

In [ ]:
pred=model.predict(X_test)

In [ ]:
print(accuracy_score(y_test, pred))

In [ ]:
x=confusion_matrix(y_test,pred)

In [ ]:
sns.heatmap(x, annot=True, fmt='d')

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
select_feature= SelectKBest(chi2,k=5).fit(X_train,y_train)
k=select_feature.scores_
a={'score':k,'Name':X_train.columns}
b=pd.DataFrame(a)
print(b)                 

Best 5 feature to classify is that area_mean, area_se, texture_mean, concavity_worst and concavity_mean. So lets se what happens if we use only these best scored 5 feature.

In [ ]:
x_train_2 = select_feature.transform(X_train)
x_test_2 = select_feature.transform(X_test)
#random forest classifier with n_estimators=10 (default)
clf_rf_2 = RandomForestClassifier()      
clr_rf_2 = clf_rf_2.fit(x_train_2,y_train)
ac_2 = accuracy_score(y_test,clf_rf_2.predict(x_test_2))
print('Accuracy is: ',ac_2)
cm_2 = confusion_matrix(y_test,clf_rf_2.predict(x_test_2))
sns.heatmap(cm_2,annot=True,fmt="d")